In [ ]:
import matplotlib
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib import ticker
from scipy.stats import circmean, circstd

from fit_diagnostics import abs_residuals
from plot_utils import antpos_map, plot_res, plot_res_grouped, plot_res_heatmap
from red_likelihood import makeCArray
from red_utils import find_rel_df, flt_ant_coords, split_rel_results

In [ ]:
%matplotlib inline

In [ ]:
jd_time = 2458098.43869
pol = 'ee'
dist = 'cauchy'
dir_path = '.'

In [ ]:
rel_df = find_rel_df(jd_time, pol, dist, dir_path)

In [ ]:
with open('rel_df.{}.{}.md.pkl'.format(jd_time, pol), 'rb') as f:
    md = pickle.load(f)
    
df = pd.read_pickle(rel_df)
df

In [ ]:
# check results for a given frequency & time integration
test_freq = 600
test_tint = 0

resx = df.loc[(test_freq, test_tint)][5:-2].values.astype(float)
test_vis, test_gains = split_rel_results(resx, md['no_unq_bls'])
test_gamps = np.abs(test_gains)
test_gphases = np.angle(test_gains)
mean_test_gamps = np.mean(test_gamps)
mean_test_gphases = np.mean(test_gphases)

print('Gain amplitude mean is: {}'.format(mean_test_gamps))
print('Gain phase mean is: {}'.format(mean_test_gphases))

In [ ]:
antpos = flt_ant_coords(jd_time, md['antpos'])

In [ ]:
antpos_map(test_gamps, antpos, 'Gain amplitudes', center=1)
print('Gain amplitude solutions for frequency channel {} and time integration {} are: '\
      '\n{}'.format(test_freq, test_tint, test_gamps))

In [ ]:
antpos_map(test_gphases, antpos, 'Gain phases')
print('Gain phase solutions are:\n{}'.format(test_gphases))

In [ ]:
# success rate percentage
success_pct = df['success'].sum() / df['success'].size * 100
print('{}% of iterations (freq/tint slice) were succesful'.format(round(success_pct, 2)))

### Number of iterations required

In [ ]:
plot_res(df, 'nit', clip=True)

In [ ]:
plot_res_grouped(df, 'nit', logy=True)

In [ ]:
plot_res_heatmap(df, 'nit', clip=True)

### Log-likelihood

In [ ]:
plot_res(df, 'fun', clip=True)

In [ ]:
plot_res_grouped(df, 'fun', logy=True)

In [ ]:
plot_res_heatmap(df, 'fun', clip=True)

### Residuals

Look at both the real and imaginary of the the median absolute normalized residual

In [ ]:
df[['med_abs_norm_res_Re', 'med_abs_norm_res_Im']] = df.apply(lambda row: \
pd.Series(abs_residuals(row['norm_residual'])), axis=1)

In [ ]:
plot_res_heatmap(df, 'med_abs_norm_res_Re', clip=True)

In [ ]:
plot_res_heatmap(df, 'med_abs_norm_res_Im', clip=True)

### Stability of gains, visibility solutions and degenerate parameters

#### Gains

In [ ]:
no_ants = md['no_ants']
no_resid_p = 4 # number of residual parameters in df dataframe
gains_df = df.iloc[:, -no_ants*2-no_resid_p:-no_resid_p]

gains_df['gamp_mean'] = gains_df.apply(lambda row: \
                        np.mean(np.abs(makeCArray(row[:no_ants*2].values))), \
                        axis=1)
gains_df['gphase_mean'] = gains_df.apply(lambda row: \
                          np.mean(np.angle(makeCArray(row[:no_ants*2].values))), \
                          axis=1)

In [ ]:
# expect all of these to be 1, since the gain amplitudes are either constrained or 
# normalized
plot_res(gains_df, 'gamp_mean', ylim=(0.9, 1.1))

In [ ]:
plot_res(gains_df, 'gphase_mean', ylim=(-np.pi, np.pi))

In [ ]:
plot_res_heatmap(gains_df, 'gphase_mean')

##### Gain stability at test_freq {{test_freq}}

In [ ]:
gainsC_df = gains_df.iloc[:, :-2].apply(lambda row: makeCArray(row.values), axis=1)
gainsC_df = pd.DataFrame(gainsC_df.values.tolist(), index=gainsC_df.index)

gamps_df = gainsC_df.abs()
gphases_df = gainsC_df.apply(np.angle)
gamps_df.columns = np.arange(no_ants) + 1
gphases_df.columns = gamps_df.columns

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
ax = sns.heatmap(gamps_df.loc[test_freq, :])
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_title('Gain amplitudes for frequency channel {}'.format(test_freq))
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
ax = sns.heatmap(gphases_df.loc[test_freq, :])
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_title('Gain phases for frequency channel {}'.format(test_freq))
plt.tight_layout()

##### Gain stability across frequencies

In [ ]:
std_rng = 3

fig, ax = plt.subplots(figsize=(11, 7))
# mean over the time integrations for each frequency
avg_gamps_tint = gamps_df.groupby(level=0).mean()
piv = pd.pivot_table(avg_gamps_tint, columns='freq')
vrng = np.ceil(np.std(piv.values)*std_rng*10)/10
ax = sns.heatmap(piv, center=1, cmap='bwr', vmin=1-vrng, vmax=1+vrng)
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=-50))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_title('Mean (over time integrations) gain amplitudes')
plt.tight_layout()

In [ ]:
center = 0
fig, ax = plt.subplots(figsize=(11, 7))
avg_gphases_tint = gphases_df.groupby(level=0)
avg_gphases_tint = gphases_df.groupby(level=0).mean()
piv = pd.pivot_table(avg_gphases_tint, columns='freq')
ax = sns.heatmap(piv, center=center, cmap='bwr', vmin=-np.pi, vmax=np.pi)
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=-50))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_title('Mean (over time integrations) gain phases')
plt.tight_layout()

In [ ]:
high_circ = np.pi
low_circ = -np.pi

center = 0
fig, ax = plt.subplots(figsize=(11, 7))
avg_gphases_tint = gphases_df.groupby(level=0)
circ_res = avg_gphases_tint.apply(lambda x: circmean(x, low=low_circ, high=high_circ, \
                                                     axis=0))
avg_gphases_tint = pd.DataFrame(circ_res.values.tolist(), index=circ_res.index)
piv = pd.pivot_table(avg_gphases_tint, columns='freq')
ax = sns.heatmap(piv, center=center, cmap='bwr', vmin=-np.pi, vmax=np.pi)
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=-50))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_title('Circular mean (over time integrations) of gain phases')
plt.tight_layout()

##### Average gains across all frequencies and time integrations

In [ ]:
avg_gamps = gamps_df.mean(axis=0).values
avg_gphases = gphases_df.mean(axis=0).values
circ_mean_gphases = gphases_df.apply(lambda col: circmean(col, low=low_circ, \
                                                          high=high_circ), axis=0).values

In [ ]:
antpos_map(avg_gamps, antpos, 'Mean gain amplitudes', center=1)

In [ ]:
antpos_map(avg_gphases, antpos, 'Mean gain phases', center=0)

In [ ]:
antpos_map(circ_mean_gphases, antpos, 'Circular mean of gain phases', center=0)

##### Standard deviation of gains across all frequencies and time integrations

In [ ]:
std_gamps = gamps_df.std(axis=0)
std_gphases = gphases_df.std(axis=0)
circ_std_gphases = gphases_df.apply(lambda col: circstd(col), axis=0).values

In [ ]:
antpos_map(std_gamps, antpos, 'Standard deviation of gain amplitudes', cmap='Oranges')

In [ ]:
antpos_map(std_gphases, antpos, 'Standard deviation of gain phases', cmap='Oranges')

In [ ]:
antpos_map(circ_std_gphases, antpos, 'Circular standard deviation of gain phases', \
           cmap='Oranges')

#### Visibility solutions

In [ ]:
no_unq_bls = md['no_unq_bls']
no_min_p = 5 # number of columns in df that are attributes of the SciPy OptimizeResult 
vis_df = df.iloc[:, no_min_p:no_unq_bls*2+no_min_p]

vis_df['vamp_mean'] = vis_df.apply(lambda row: \
                      np.mean(np.abs(makeCArray(row[:no_unq_bls*2].values))), \
                      axis=1)
vis_df['vphase_mean'] = vis_df.apply(lambda row: \
                        np.mean(np.angle(makeCArray(row[:no_unq_bls*2].values))), \
                        axis=1)

In [ ]:
plot_res(vis_df, 'vamp_mean', clip=True, clip_pctile=98)

In [ ]:
plot_res_heatmap(vis_df, 'vamp_mean', clip=True, clip_pctile=98)

In [ ]:
plot_res(vis_df, 'vphase_mean', clip=False, ylim=(-np.pi, np.pi))

In [ ]:
plot_res_heatmap(vis_df, 'vphase_mean')

##### Visibilities across baselines at test_integration {{test_tint}}

In [ ]:
visC_df = vis_df.iloc[:, :-2].apply(lambda row: makeCArray(row.values), axis=1)
visC_df = pd.DataFrame(visC_df.values.tolist(), index=visC_df.index)
visC_tint_df = visC_df.loc[pd.IndexSlice[:, test_tint], :].droplevel(level=1)

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
piv = pd.pivot_table(np.abs(visC_tint_df), columns='freq')
vmax = np.nanpercentile(piv.values, 98)
ax = sns.heatmap(piv, vmax=vmax)
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=-50))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_title('Visibility amplitudes for time_integration {}'.format(test_tint))
plt.tight_layout()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 7))
piv = pd.pivot_table(visC_tint_df.apply(np.angle), columns='freq')
ax = sns.heatmap(piv, center=0, cmap='bwr')
ax.xaxis.set_major_locator(ticker.MultipleLocator(50))
ax.xaxis.set_major_formatter(ticker.ScalarFormatter(useOffset=-50))
ax.yaxis.set_major_locator(ticker.MultipleLocator(5))
ax.yaxis.set_major_formatter(ticker.ScalarFormatter())
ax.set_title('Visibility phases for time_integration {}'.format(test_tint))
plt.tight_layout()